In [2]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [6]:
df_procesada = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")
df_procesada["FECHA 1º evaluación"] = pd.to_datetime(
    df_procesada["FECHA 1º evaluación"], format="%Y-%m-%d"
)

In [17]:
df_procesada.groupby("ANIO_PRIMERA_EVALUACION")[
    "DIAGNOSTICO PRINCIPAL"
].value_counts().to_frame().to_excel("prueba.xlsx")